# Lecture 05 -- DDL, DML & Constraints

In [1]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [2]:
%reload_ext sql

There's a new jupysql version available (0.10.12), you're running 0.10.0. To upgrade: pip install jupysql --upgrade
Deploy Flask apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [3]:
!createdb lecture5 -h localhost

In [4]:
%sql postgresql://127.0.0.1:5432/lecture5

# manager demo

Creating the manager relation

In [13]:
%sql DROP TABLE IF EXISTS managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

++
||
++
++

In [14]:
%%sql
CREATE TABLE managers (
    manager_name VARCHAR(20),
    age INTEGER DEFAULT 50,
    address TEXT,
    manager_id CHAR(4),
    PRIMARY KEY (manager_id),
    UNIQUE (manager_name)
);


/***********/
SELECT * FROM managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

manager_name,age,address,manager_id


<br/><br/><br/>

Add a tuple (success)

In [16]:
%%sql
INSERT INTO managers
    (manager_id, age, manager_name)
VALUES
    (1, 23, 'Lisa'),
    (2, 23, 'Michael')
;

/***********/
SELECT * FROM managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

2 rows affected.

manager_name,age,address,manager_id
Lisa,23,None,1
Michael,23,None,2


<br/><br/><br/>

Add a tuple (failure)

In [17]:
%%sql
INSERT INTO managers
    (manager_name, age)
VALUES
    ('Rich', 123);

/***********/
SELECT * FROM managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

RuntimeError: (psycopg2.errors.NotNullViolation) null value in column "manager_id" of relation "managers" violates not-null constraint
DETAIL:  Failing row contains (Rich, 123, null, null).

[SQL: INSERT INTO managers
    (manager_name, age)
VALUES
    ('Rich', 123);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
If you need help solving this issue, send us a message: https://ploomber.io/community


<br/><br/><br/>

Change schema: add attributes

In [20]:
%%sql
ALTER TABLE managers
    DROP address,
    -- DROP income,
    ADD address VARCHAR (20),
    ADD income REAL DEFAULT 1000000.0;

/***********/
SELECT * FROM managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

manager_name,age,manager_id,address,income
Lisa,23,1,None,1000000.0
Michael,23,2,None,1000000.0


What if we want to add a timestamp?

In [22]:
%%sql
ALTER TABLE managers
    ADD created_at timestamptz,
    ADD started_at timestamptz;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

++
||
++
++

### Notes About Times And Timestamps:

* A `timestamp` is a date + time
* A time _without a timestamp_ **does not mean anything** without a bunch of context in many cases...
* _Always_ (almost) use `timestamptz` over `timestamp`
* There is also `time with timezone` and `time [without timezone]` type, but don't use the later...

In [26]:
%%sql
INSERT INTO managers(manager_id, created_at, started_at)
VALUES
    (10, '2024-08-20T09:30:00.000-0700', now())
;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

1 rows affected.

++
||
++
++

In [27]:
%sql SELECT * FROM managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

3 rows affected.

manager_name,age,manager_id,address,income,created_at,started_at
Lisa,23,1,None,1000000.0,None,None
Michael,23,2,None,1000000.0,None,None
None,50,10,None,1000000.0,2024-08-20 16:30:00+00:00,2024-09-12 08:58:46.893113+00:00


<br/><br/><br/>

Change schema: drop attributes

In [29]:
%%sql
ALTER TABLE managers
DROP address,
DROP income;

/***********/
SELECT * FROM managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

3 rows affected.

manager_name,age,manager_id,created_at,started_at
Lisa,23,1,None,None
Michael,23,2,None,None
None,50,10,2024-08-20 16:30:00+00:00,2024-09-12 08:58:46.893113+00:00


<br/><br/><br/>

Delete the relation entirely

In [30]:
%%sql
DROP TABLE managers;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

++
||
++
++

In [31]:
%sql
/***********/
SELECT * FROM managers;

SyntaxError: invalid syntax (167872053.py, line 2)

In [32]:
%%sql
DROP TABLE IF EXISTS Stops;
CREATE TABLE Stops(
  stop_id INTEGER,
  person_id INTEGER,
  -- This is not great...we should have a timezone, but the original data did not.
  stop_time TIMESTAMP,
  race VARCHAR(10),
  location VARCHAR(20) NOT NULL,
  age INTEGER,
  arrest BOOLEAN DEFAULT False,
  PRIMARY KEY (stop_id),
  UNIQUE (person_id, stop_time)
  );


Running query in 'postgresql://127.0.0.1:5432/lecture5'

++
||
++
++

In [33]:
%sql SELECT * FROM Stops;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

stop_id,person_id,stop_time,race,location,age,arrest


In [34]:
%%sql
INSERT INTO Stops
    (stop_id, person_id, stop_time, location, arrest)
VALUES
    -- (0543, 1234, '2023-09-12 13:43:00', 'Oakland')
    (9999, 8888, '2023-09-12 13:43:00', 'Oakland', NULL)
;
SELECT * FROM Stops;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

1 rows affected.

1 rows affected.

stop_id,person_id,stop_time,race,location,age,arrest
9999,8888,2023-09-12 13:43:00,None,Oakland,None,None


## IMDB Datebase

In [35]:
%reload_ext sql

In [36]:
%sql postgresql://127.0.0.1:5432/lecture5

In [38]:
%%sql
DROP TABLE IF EXISTS actors CASCADE;
DROP TABLE IF EXISTS movies CASCADE;
DROP TABLE IF EXISTS cast_info;

CREATE TABLE actors (
  id INTEGER,
  name TEXT,
  PRIMARY KEY(id)
);

CREATE TABLE movies (
  id INTEGER,
  title TEXT,
  PRIMARY KEY(id)
);

CREATE TABLE cast_info (
  person_id INTEGER,
  movie_id INTEGER,
  FOREIGN KEY (person_id)
    REFERENCES actors (id)
    ON DELETE SET NULL
    ON UPDATE CASCADE,
  FOREIGN KEY (movie_id)
    REFERENCES movies (id)
    ON DELETE SET NULL);

INSERT INTO actors VALUES
    (1, 'Tom Hanks'),
    (3, 'Michelle Yeoh')
;

INSERT INTO movies VALUES
    (23, 'Forrest Gump'),
    (45, 'Tomorrow Never Dies')
;

INSERT INTO cast_info VALUES
    (1, 23),
    (3, 45)
;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

2 rows affected.

2 rows affected.

++
||
++
++

In [39]:
%%sql
SELECT * FROM cast_info;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

person_id,movie_id
1,23
3,45


A. What if we… Delete a tuple from Actor corresponding to id = 1?

In [40]:
%%sql
DELETE FROM actors
WHERE id = 1;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

1 rows affected.

++
||
++
++

In [41]:
%%sql
SELECT * FROM cast_info;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

person_id,movie_id
3,45
None,23


(note: reset table layout for each choice)

B. Change a tuple in Actor from id = 1 to id = 2?

In [42]:
%%sql
UPDATE actors
SET id = 2
WHERE id = 1;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

++
||
++
++

In [43]:
%%sql
SELECT * FROM actors;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

1 rows affected.

id,name
3,Michelle Yeoh


In [44]:
%%sql
SELECT * FROM cast_info;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

person_id,movie_id
3,45
None,23


C. Delete a tuple from Movie corresponding to id = 23?

In [45]:
%%sql
DELETE FROM movies
WHERE id = 23;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

1 rows affected.

++
||
++
++

In [46]:
%%sql
SELECT * FROM cast_info;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

2 rows affected.

person_id,movie_id
3,45
None,None


D. Change a tuple in Movie from id = 23 to id = 24?

In [47]:
%%sql
UPDATE movies
SET id = 24
WHERE id = 23;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

++
||
++
++

E. Insert a tuple into `cast_info` that adds a new person_id not found in Actor?

In [48]:
%%sql
INSERT INTO cast_info VALUES
    (467, 23)
;

Running query in 'postgresql://127.0.0.1:5432/lecture5'

RuntimeError: (psycopg2.errors.ForeignKeyViolation) insert or update on table "cast_info" violates foreign key constraint "cast_info_person_id_fkey"
DETAIL:  Key (person_id)=(467) is not present in table "actors".

[SQL: INSERT INTO cast_info VALUES
    (467, 23)
;]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
If you need help solving this issue, send us a message: https://ploomber.io/community
